In [1]:
import os
import json

# List result Files
experiment_files = [a for a in os.listdir("./out") if ".json" in a]

# Load result Files
all_experiments = []
for filename in experiment_files:
    with open("./out/{}".format(filename), "r") as f:
        all_experiments.append(json.load(f))

# Aglomerate similar results
unique_experiment_names = list(dict.fromkeys([a["model"] for a in all_experiments]))
print(unique_experiment_names)

sorted_experiments = {}
for e in unique_experiment_names:
    sorted_experiments[e] = []

for e in all_experiments:
    sorted_experiments[e['model']].append(e)

['lee2021_layer_10', 'matsubara2022_5', 'lee2021_layer_5', 'lee2021_layer_7', 'matsubara2022_4', 'matsubara2022_1', 'dummy', 'lee2021_layer_3', 'matsubara2022_2', 'matsubara2022_3']


In [2]:
baseline = sorted_experiments['dummy']

total_energy = sum([b['joules1'] for b in baseline])
total_seconds = sum([b['seconds'] for b in baseline])
baseline_power = total_energy/total_seconds
print(baseline_power)

2.9698283918805557


In [3]:
aggregated_experiments = []

for exp_name, exp_list in sorted_experiments.items():
    joules = sum([e['joules1'] for e in exp_list])
    seconds = sum([e['seconds'] for e in exp_list])
    corrected_joules = joules - baseline_power*seconds
    num_images = sum([e['num_images'] for e in exp_list])
    aggregated_experiments.append({
        "joules": joules,
        "seconds": seconds,
        "corrected_joules": joules - baseline_power*seconds,
        "corrected_joules_per_image": corrected_joules / num_images,
        "inference_time": seconds / num_images,
        "num_images": num_images,
        "model": exp_name,
        "macs": exp_list[0]['macs'],
        "macs": exp_list[0]['macs'],
        "map": exp_list[0]['map'],
        "params": exp_list[0]['params'],
    })
    
with open("test.json", "w+") as f:
    json.dump(aggregated_experiments, f)

In [4]:
import pandas as pd
df = pd.read_json('test.json')
df = df.set_index("model")
df = df.sort_index(ascending=True)
print(df)

                       joules  seconds  corrected_joules  \
model                                                      
dummy             1069.138221      360          0.000000   
lee2021_layer_10  1449.352688      360        380.214467   
lee2021_layer_3    476.737904      120        120.358497   
lee2021_layer_5    716.624028      180        182.054918   
lee2021_layer_7    966.204086      240        253.445272   
matsubara2022_1    973.729497      240        260.970683   
matsubara2022_2    494.501375      120        138.121968   
matsubara2022_3    747.987091      180        213.417981   
matsubara2022_4   1242.179548      300        351.231030   
matsubara2022_5    974.028906      240        261.270092   

                  corrected_joules_per_image  inference_time  num_images  \
model                                                                      
dummy                               0.000000        0.714286         504   
lee2021_layer_10                    0.647725       